In [1]:
print('Validate nvidia-smi (software managemtn interface):')
!nvidia-smi
print()
print('Validate nvcc --version:')
!nvcc --version

Validate nvidia-smi (software managemtn interface):
Sat Oct 21 09:54:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P8     7W /  N/A |     99MiB /  3910MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                    

In [9]:
import pandas as pd
import os

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0
main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
# takt last 5% of sequences of data
last_5pct = times [-int(0.05*len(times))]
print("last_5pct:")
print(last_5pct)
# We take 5% of validating data
validation_main_df = main_df[(main_df.index >= last_5pct)]
print ('validation_main_df:')
print (validation_main_df)
main_df = main_df[(main_df.index < last_5pct)]
print ('main_df:')
print (main_df)

last_5pct:
1534922100
validation_main_df:
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1534922100    6684.500000        0.969366      57.509998       66.463028   
1534922160    6684.500000        0.611018      57.509998        3.616516   
1534922220    6682.740234        1.121768      57.509998       13.260421   
1534922280    6682.660156        0.912729      57.509998       19.851404   
1534922340    6682.450195        0.334119      57.509998       17.104265   
...                   ...             ...            ...             ...   
1535214960    6713.140137        0.769891      58.020000        6.434783   
1535215020    6714.520020        1.002652      58.009998        7.301921   
1535215080    6714.520020        1.021925      58.020000       23.802017   
1535215140    6715.000000        3.645508      58.020000        6.953497   
1535215200    6715.000000        0.513560     

In [11]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    # We drop the 'future' column. We only need target column for neural network.
    df = df.drop('future', axis=1)
    # We scale the column
    for col in df.columns:
        if col != "target":
            # We normalize with percent change with current row and previous row.
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            # scale is from preprocessong of sciki-learn 
            # Scale the current value with (max-min)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    # pop out the old iterm. We only keep max queue of 60 minutes.
    prev_days = deque(maxlen=SEQ_LEN)
    print('df.head():')
    print(df.head())
    print('df.columns.shape:')
    print(df.columns.shape)
    for c in df.columns:
        print ('c:', c)

    print('df.values.shape:')
    print(df.values.shape)
    print('df.values[:1]')
    print(df.values[:1])
    
main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
last_5pct = times [-int(0.05*len(times))]
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]
# We have split the data into model and validation.
# we ned to balance and scale.
preprocess_df (main_df)
#train_x, train_y = preprocess_df (main_df)
#validation_x, validation_y = preprocess_df (validation_main_df)

df.head():
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528969140      -0.002513       -0.078919      -0.540429       -0.117090   
1528969200      -0.229992        0.003410       0.004048       -0.118307   
1528969260      -0.000830       -0.074039       0.004048       -0.072997   
1528969320       0.116299       -0.049626       0.004048       -0.108241   
1528969380       0.000937       -0.082052       0.004048       -0.048516   

            ETH-USD_close  ETH-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1528969140       0.004601       -0.047838       0.011062       -0.006217   
1528969200       0.004601       -0.045377       0.257552       -0.005884   
1528969260       0.021273       -0.032900       0.265415       -0.006200   
1528969320       0.004601       -0.049530       0.455951       -0.005898   


In [37]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install scikit-learn
from collections import deque
import random
import numpy as np # pip install numpy

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3 # We predict 3 min.
RATIO_TO_PREDICT = "LTC-USD"

def classify (current, future):
    if float(future) > float(current):
        return 1   # 1 is good, you can buy it.
    else:
        return 0

def preprocess_df(df):
    df = df.drop('future', axis=1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)  # drop n/a again after scale.        
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    # print('len(prev_days):', len(prev_days))
    # len(prev_days): 0
    # We iterate for all the columns (values):
    #i_cnt = 0
    for i in df.values:
        # append all 60 rows (60 minutes) in datasets.
        # Stoped before last i (-1) which is column 'target'.
        prev_days.append([n for n in i[:-1]])
        if len (prev_days) == SEQ_LEN:
            # only the first 0-59 is not stored.
            # queue always maintain maxiumn 60 elements.
            sequential_data.append([np.array(prev_days), i[-1]]) 
            #i_cnt += 1
    # print('i_cnt:', i_cnt)
    # i_cnt: 82178
    print('len(df):', len(df))
    # len(df): 82237
    print('df.shape:', df.shape)
    # df.shape: (82237, 9)
    print('df.values.shape:', df.values.shape)
    print('df.columns.shape:', df.columns.shape)
    print('df.values.shape:', df.values.shape)
    print('len(prev_days):', len(prev_days))
    print('len(sequential_data):', len(sequential_data))
    # len(sequential_data): 82178
    random.shuffle(sequential_data)

main_df = pd.DataFrame()
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, 
                 names=["time", "low", "high",  "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT) 
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df['future']))
times = sorted (main_df.index.values)
last_5pct = times [-int(0.05*len(times))]
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]
# We have split the data into model and validation.
# we ned to balance and scale.
preprocess_df (main_df)
#train_x, train_y = preprocess_df (main_df)
#validation_x, validation_y = preprocess_df (validation_main_df)

len(df): 82237
df.shape: (82237, 9)
df.values.shape: (82237, 9)
df.columns.shape: (9,)
df.values.shape: (82237, 9)
len(prev_days): 60
len(sequential_data): 82178
